# Using CARTOframes to view earthquakes from the last 30 days

This notebook demonstrates how to make an animated map using CARTOframes with a time column. The notebook is viewable here: http://nbviewer.jupyter.org/github/CartoDB/cartoframes/blob/more-examples/examples/Demo%20%28Earthquakes%29.ipynb

In [1]:
import cartoframes
from cartoframes import Credentials
from cartoframes import Layer, BaseMap, styling
import pandas as pd
%matplotlib inline

In [8]:
USERNAME = 'michellemho-carto'  # <-- replace with your username 
APIKEY = 'abcde'       # <-- your CARTO API key
creds = Credentials(username=USERNAME, 
                    key=APIKEY)
creds.save()  # save credentials for later use (not dependent on Python session)
cc = cartoframes.CartoContext(creds)

In [2]:
cc = cartoframes.CartoContext()

In [4]:
# Use Pandas to read a CSV from the USGS feed of earthquakes from the last 30 days
earthquakes = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')
earthquakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2017-11-13T17:52:27.630Z,37.603001,-118.818832,3.55,1.11,md,13.0,73.0,0.01531,0.03,...,2017-11-13T18:42:03.539Z,"14km ESE of Mammoth Lakes, California",earthquake,0.36,0.65,0.160,11.0,automatic,nc,nc
1,2017-11-13T17:48:21.960Z,34.316333,-118.147500,0.28,2.06,ml,9.0,112.0,0.35120,0.27,...,2017-11-13T18:24:41.082Z,"12km NE of La Crescenta-Montrose, CA",earthquake,0.92,31.61,0.336,34.0,automatic,ci,ci
2,2017-11-13T17:46:17.648Z,62.033800,-152.867600,3.00,2.00,ml,NaN,NaN,NaN,0.84,...,2017-11-13T17:50:14.467Z,"147km WSW of Talkeetna, Alaska",earthquake,NaN,0.40,NaN,NaN,automatic,ak,ak
3,2017-11-13T17:32:09.277Z,60.577400,-142.652000,19.50,1.40,ml,NaN,NaN,NaN,0.27,...,2017-11-13T17:35:55.445Z,"58km NNW of Cape Yakataga, Alaska",earthquake,NaN,0.10,NaN,NaN,automatic,ak,ak
4,2017-11-13T17:29:38.770Z,33.187333,-115.638500,3.46,1.19,ml,13.0,150.0,0.01400,0.21,...,2017-11-13T17:33:15.529Z,"13km WSW of Niland, CA",earthquake,0.89,0.42,0.204,20.0,automatic,ci,ci


In [12]:
# Write earthquake data to CARTO
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

The following columns were changed in the CARTO copy of this dataframe:
magType -> magtype
horizontalError -> horizontalerror
depthError -> deptherror
magError -> magerror
magNst -> magnst
locationSource -> locationsource
magSource -> magsource
Table successfully written to CARTO: https://michellemho-carto.carto.com/dataset/earthquakes_30


In [5]:
# Map the earthquake data
cc.map(layers=[Layer('earthquakes_30')])

In [6]:
# Style the color and size by the magnitude
cc.map(layers=[Layer('earthquakes_30',
       color={'column': 'mag', 'scheme': styling.redOr(5, bin_method='equal')},
       size={'column':'mag', 'bins':5, 'bin_method':'equal'})])

In [15]:
# Try torque animation of time-stamp column

# Just to make sure time-stamp column is a pandas date type
earthquakes['time'] = pd.to_datetime(earthquakes.time)

In [16]:
earthquakes.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2017-11-09 14:03:55.926,64.614300,-149.396800,19.50,1.60,ml,NaN,NaN,NaN,0.66,...,2017-11-09T14:12:34.486Z,"14km WNW of North Nenana, Alaska",earthquake,NaN,0.10,NaN,NaN,automatic,ak,ak
1,2017-11-09 13:27:52.200,35.845833,-118.257167,1.69,1.27,ml,15.0,99.0,0.1504,0.18,...,2017-11-09T13:31:25.279Z,"18km NE of Kernville, CA",earthquake,0.4,0.81,0.21,15.0,automatic,ci,ci


In [ ]:
# Update CARTO dataset
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

The following columns were changed in the CARTO copy of this dataframe:
magType -> magtype
horizontalError -> horizontalerror
depthError -> deptherror
magError -> magerror
magNst -> magnst
locationSource -> locationsource
magSource -> magsource


In [7]:
# Create an animated map by specifying the time of the layer
cc.map(layers=[Layer('earthquakes_30', time = {'column':'time', 'method':'sum', 'trails':4, 'duration':30})])